# Develop a CasADi Model of The Steady-State Solar Plant RTO Problem

Replicates the calculations in the following Excel spreadsheet and uses CasADi to solve the optimization problem.

- `SS Solar Plant Optimization New Thermal & Flow Model 15 DVs I-O 2025-10-12a.xlsm`

In [ ]:
import casadi as cas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Import functions from the solar_plant_rto module
from problems.solar_plant_rto.solar_plant_gen_rto import (
    calculate_pump_and_drive_efficiency,
    calculate_pump_dp,
    calculate_pump_fluid_power,
    calculate_collector_flow_rate,
    calculate_collector_oil_exit_temp,
    solar_plant_rto_solve,
    solar_plant_gen_rto_solve
)

## Test Data

In [ ]:
from tests.test_solar_plant_gen_rto import test_data

# This data is from the Excel spreadsheet
test_data

In [ ]:
m_pumps = test_data['m_pumps']
n_lines = test_data['n_lines']

## Pump Characteristics

In [ ]:
flow_rates = np.linspace(50, 120, 71)

fig, axes = plt.subplots(3, 1, sharex=True, figsize=(7, 5.5))

for pump_speed in [1500, 2000, 2500, 3000]:
    efficiency = calculate_pump_and_drive_efficiency(
        flow_rates, pump_speed
    )
    delta_p = calculate_pump_dp(
        pump_speed, flow_rates, m_pumps
    )
    power = calculate_pump_fluid_power(flow_rates, delta_p)

    ax = axes[0]
    ax.plot(flow_rates, delta_p, label=pump_speed)
    ax.set_ylabel('dP (kPa)')
    ax.grid(True)
    ax.legend(title="Speed (rpm)")
    ax.set_title("Pump Curves")


    ax = axes[1]
    ax.plot(flow_rates, efficiency * 100, label=pump_speed)
    ax.set_ylabel('Efficiency (%)')
    ax.grid(True)
    ax.legend(title="Speed (rpm)")
    ax.set_title("Efficiency Curves")

    ax = axes[2]
    ax.plot(flow_rates, power, label=pump_speed)
    ax.set_xlabel('Flow rate (kg/s)')
    ax.set_ylabel('Power (kW)')
    ax.grid(True)
    ax.legend(title="Speed (rpm)")
    ax.set_title("Power Curves")

plt.tight_layout()
plt.show()

## Collector Line Flow Characteristics

In [ ]:
test_data['oil_return_temp']

In [ ]:
test_data['loop_dp']

In [ ]:
valve_positions = np.linspace(0.2, 1.0, 41)

fig, ax = plt.subplots(figsize=(7, 2.5))

for delta_p in [1.5, 2.0, 2.5]:
    flow_rates = calculate_collector_flow_rate(
        valve_positions, delta_p, sqrt=np.sqrt
    )

    ax.plot(valve_positions, flow_rates, label=delta_p)
    ax.set_xlabel('Valve position (0-1)')
    ax.set_ylabel('Flow rate ((kg/s))')
    ax.grid(True)
    ax.legend(title="Pressure drop (kPa)")
    ax.set_title("Collector Loop Flow Curves")

plt.tight_layout()
plt.show()

## Oil Exit Temperature Calculations

In [ ]:
test_data['oil_exit_temps'][0], test_data['collector_flow_rates'][0], test_data["loop_thermal_efficiencies"][0]

In [ ]:
# X-axis: flow rate per line (kg/s)
flow_rates = np.linspace(80, 120, 41) / 15  # per line

# Plot curves
oil_return_temps = [240, 250, 260, 270, 290]  # degC
ambient_temps = [15, 20, 25, 20, 35]  # degC
solar_rates = [500, 600, 700, 800, 900]  # W/m2
loop_thermal_efficiency = 0.9

fig, axes = plt.subplots(3, 1, sharex=True, figsize=(7, 5.5))

ax = axes[0]
for solar_rate in solar_rates:
    oil_exit_temps = calculate_collector_oil_exit_temp(
        flow_rates,
        oil_return_temps[2],
        ambient_temps[2],
        solar_rate,
        loop_thermal_efficiency,
        exp=np.exp,
        pi=np.pi
    )
    ax.plot(flow_rates, oil_exit_temps, label=solar_rate)
ax.set_ylabel('°C')
ax.grid(True)
ax.legend(title="$I_C$ ($W/m^2$)")
ax.set_title("Collector Loop Exit Temperature vs. Solar Rate")
ax.set_xlabel('Flow rate (kg/s)')

ax = axes[1]
for ambient_temp in ambient_temps:
    oil_exit_temps = calculate_collector_oil_exit_temp(
        flow_rates,
        oil_return_temps[2],
        ambient_temp,
        solar_rates[2],
        loop_thermal_efficiency,
        exp=np.exp,
        pi=np.pi
    )
    ax.plot(flow_rates, oil_exit_temps, label=ambient_temp)
ax.set_ylabel('°C')
ax.grid(True)
ax.legend(title="$T_{amb}$ (deg C)")
ax.set_title("Collector Loop Exit Temperature vs. Ambient Temperature")
ax.set_xlabel('Flow rate (kg/s)')


ax = axes[2]
for oil_return_temp in oil_return_temps:
    oil_exit_temps = calculate_collector_oil_exit_temp(
        flow_rates,
        oil_return_temp,
        ambient_temps[2],
        solar_rates[2],
        loop_thermal_efficiency,
        exp=np.exp,
        pi=np.pi
    )
    ax.plot(flow_rates, oil_exit_temps, label=oil_return_temp)
ax.set_ylabel('°C')
ax.grid(True)
ax.legend(title="$T_{return}$ (deg C)")
ax.set_title("Collector Loop Exit Temperature vs. Oil Return Temperature")
ax.set_xlabel('Flow rate (kg/s)')

plt.tight_layout()
plt.show()

## Solar Collector Lines and Pumps RTO Model

In [ ]:
# Maximizes net potential energy generation

ambient_temp = 20  # degC
solar_rates = np.linspace(500, 900, 41)  # W/m2
oil_return_temp = 270  # degC

# Silence solver output
solver_opts = {'ipopt.tol': 1e-9, 'print_time': False, 'ipopt': {'print_level': 0}}

valve_positions = []
oil_flow_rates = []
oil_exit_temps = []
index_values = []

for solar_rate in tqdm(solar_rates):

    sol, variables = solar_plant_rto_solve(
        solar_rate,
        ambient_temp,
        oil_return_temp,
        m_pumps,
        n_lines,
        solver_opts=solver_opts
    )

    valve_positions.append(variables['valve_positions'])
    oil_flow_rates.append(variables['collector_flow_rates'])
    oil_exit_temps.append(variables['oil_exit_temps'])
    index_values.append(solar_rate)

valve_positions = pd.DataFrame.from_records(
    valve_positions,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
oil_flow_rates = pd.DataFrame.from_records(
    oil_flow_rates,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)
oil_exit_temps = pd.DataFrame.from_records(
    oil_exit_temps,
    index=pd.Index(index_values, name='Solar Rate (W/m2)'),
    columns=pd.Index(range(n_lines), name='Collector Line')
)


In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(7, 5.5))

ax = axes[0]
for line in oil_exit_temps[:8]:  # Plot first 8 lines only
    oil_exit_temps[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Line Exit Temperatures',
        xlabel='Solar Rate (W/m2)',
        ylabel='Oil Temperature (°C)',
    )
ax.set_ylim([390, 401])
ax.grid(True)

ax = axes[1]
for line in valve_positions[:8]:  # Plot first 8 lines only
    valve_positions[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector FCV Positions',
        xlabel='Solar Rate (W/m2)',
        ylabel='Valve Position (0-1)',
    )
ax.set_ylim([None, 1.01])
ax.grid(True)

ax = axes[2]
for line in oil_flow_rates[:8]:  # Plot first 8 lines only
    oil_flow_rates[line].plot(
        ax=ax,
        style='.-',
        title=f'Collector Oil Flow Rates',
        xlabel='Solar Rate (W/m2)',
        ylabel='Flow Rate (kg/s)',
    )
ax.grid(True)

plt.tight_layout()
plt.show()